# **Fine-tune Gemma-2-2b-it With Text-to-SQL Data**

**Test Result Version 1:**
### Evaluate by Full SQL Script Comparison
> Directly compare the string content of each `Model Answer` with the `Ground Truth Answer` to see if they are exactly the same.

| Model                   | Accuracy        |
| ----------------------- | --------------- |
| Gemma2-2b-it            | 3.33%           |
| Fine-tuned Gemma2-2b-it | **33.33%**      |

### Evaluate by SOTA LLM
> Let the SOTA LLM act as a judge, providing `SQL Context`, `Query`, `Ground Truth Answer`, and `Model Answer`. Ask the SOTA LLM to determine whether each Model Answer can achieve the same effect as the Ground Truth Answer in answering the Query.

| Model                   | Accuracy        |
| ----------------------- | --------------- |
| Gemma2-2b-it            | 23.33%          |
| Fine-tuned Gemma2-2b-it | **60.00%**      |

**References:**
- [LLM-Finetuning GitHub Repository](https://github.com/ashishpatel26/LLM-Finetuning?tab=readme-ov-file)
- [LLM Fine-tuning Chat Template](https://github.com/mst272/LLM-Dojo/tree/main/chat_template#gemma)
- [Finetune Gemma-2b for Text to SQL](https://medium.com/@hayagriva99999/finetune-gemma-2b-for-text-to-sql-90041abdda70)
- [Best Way To Fine-tune Your LLM Using a T4 GPU](https://jair-neto.medium.com/best-way-to-fine-tune-your-llm-using-a-t4-gpu-part-3-3-71c7d0514aa6)
- [Conversation on Evaluation of Text-to-SQL Task](https://github.com/explodinggradients/ragas/issues/651)
- [Steps By Step Tutorial To Fine Tune LLAMA 2 With Custom Dataset Using LoRA And QLoRA Techniques](https://www.youtube.com/watch?v=Vg3dS-NLUT4)
- [Methods and tools for efficient training on a single GPU](https://huggingface.co/docs/transformers/perf_train_gpu_one#flash-attention-2)
- [HuggingFace Developer Guides of Quantization](https://huggingface.co/docs/peft/developer_guides/quantization)
- [What Rank r and Alpha To Use in LoRA in LLM ?](https://medium.com/@fartypantsham/what-rank-r-and-alpha-to-use-in-lora-in-llm-1b4f025fd133)
- [TaskType Parameter of LoRA Config](https://discuss.huggingface.co/t/task-type-parameter-of-loraconfig/52879/6)
- [What Target Modules Should We Add When Training with LoRA](https://www.reddit.com/r/LocalLLaMA/comments/15sgg4m/what_modules_should_i_target_when_training_using/)
- [Difference of DataCollator between CausalLM and Seq2Seq Model](https://gitea.exxedu.com/aibot/LLaMA-Factory/src/commit/3a666832c119606a8d5baf4694b96569bee18659/scripts/cal_ppl.py)

## **1. Preliminary Work**

In [ ]:
!nvidia-smi

### **1-1. Install Packages**

In [ ]:
# install peft for parameter-efficient fine-tuning
!pip install -q -U peft
# install model, dataset, training booster, model evaluation, quantization from HuggingFace
!pip install -q -U transformers datasets accelerate bitsandbytes trl

# console log
print("1-1. pip install all packages and dependencies...done.")

### **1-2. Import Packages**

In [ ]:
### model training
from trl import (
    SFTConfig,
    SFTTrainer
) # use SFTConfig & SFTTrainer for easy, supervise fine-tuning
from datasets import (
    load_dataset, 
    concatenate_datasets
) # load dataset from HuggingFace and concatenate train & test dataset for data visualization
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    TrainingArguments, 
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    TextStreamer
) # load tokenizer, model, set training arguments, use quantization, create data collator and stream output text
from peft import (
    LoraConfig, 
    TaskType,
    get_peft_model,
    PeftModel
) # PEFT
from accelerate import PartialState
import numpy as np # numpy calculation
import torch # use torch.bfloat16

### visualization
import matplotlib.pyplot as plt # create charts for data visualization

### others
from tqdm import tqdm # for displaying progess bar
from typing import List # for clear typing
import pandas as pd # output excel
import random # for random sampling
import os # check if folder exists in directory or not

print("1-2. import all packages needed...done.")

### **1-3. Hyperparameters Setting**

**About Kaggle Secret**
reference:
- [Kaggle Feature Launch: User Secrets](https://www.kaggle.com/discussions/product-feedback/114053)

In [ ]:
# when execute code on Kaggle
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

# # when execute code on Localhost
# import os
# os.environ["HF_TOKEN"] = "..."
# HF_TOKEN = os.getenv("HF_TOKEN")

**TL;DR**
- To achieve performance with LoRA comparable to full finetuning, you should **train all Linear layers**. 
- Example : For LLaMA, these are: `gate_proj`, `down_proj`, `up_proj`, `q_proj`, `v_proj`, `k_proj`, and `o_proj`.

In [ ]:
try:
    DATASET = "gretelai/synthetic_text_to_sql"
    MODEL_ID = "google/gemma-2-2b-it" 
    DATASET_DIRECTORY = "/kaggle/working/"
    DEVICE = "cuda"
    SEED = 2024
    
    # train eval split
    TRAIN_SIZE = 5000
    EVAL_SIZE = 2000
    
    # collect columns we don't need while training model
    REMOVE_COLS = ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation']
    
    # hyperparameters of LoRA
    LORA_RANK = 64 # if we want to teach some new to LLM, set this not smaller than 32 is better
    LORA_ALPHA = 64 # set this the same as rank is better
    LORA_DROPOUT = 0.05 # set as the article used
    LORA_TARGET_MODULES = ["gate_proj", "down_proj", "up_proj", "q_proj", "v_proj", "k_proj", "o_proj"] # set layers we want to add LoRA, here I set all linear layers without `lm_head`
    
    # hyperparameters of Training Arguments
    OUTPUT_DIR = "text2sql_finetuning" # the directory saving our log & checkpoint model
    OPTIMIZER = "paged_adamw_8bit"
    LEARNING_RATE = 2e-4 # default
    WARMUP_RATIO = 0.03
    GRADIENT_ACCUMULUATION_STEPS = 2
    LR_SCHEDULER_TYPE = "cosine" # set as the article used
    LOGGING_STEPS = 100
    SAVE_STRATEGY = "epoch"
    PER_DEVICE_TRAIN_BATCH_SIZE = 1 # prevent out of memory
    PER_DEVICE_EVAL_BATCH_SIZE = 1 # prevent out of memory
    WEIGHT_DECAY = 0.05 # weight decay to apply to all layers except bias/LayerNorm weights
    MAX_GRAD_NORM = 0.3 # maximum fradient normal (gradient clipping)
    MAX_STEPS = 1000 # maximum number of training steps
    
    # finetuned model name (save model)
    PEFT_MODEL_ID = "lora_text2sql"
    
    # finetuned model name (if we upload our model to Kaggle/HuggingFace, we should set the name of model)
    VERSION = 1
    NEW_MODEL = "text2sql_gemma2_2b_it" # when the model is in the directory `/kaggle/working/`, use this
    KAGGLE_NEW_MODEL = "None" # when the model is uploaded to Kaggle as model, use this
    
    # LLM generation hyperparameters
    MAX_NEW_TOKENS = 1000
    TOP_P = 0.95
    TOP_K = 50
    TEMPERATURE = 0.7
    
    # how many test cases we want to evaluate
    TEST_CASES_NUMBER = 30
    
    print("1-3. setting all parameters...done.")
except Exception as e:
    print("1-3. Exception message as below:\n", e)

## **2. Load Dataset and Preprocess Data**

### **2-1. Load Dataset, Tokenizer**

In [ ]:
# Load tokenizer of Gemma-2-2b-it with HuggingFace Token
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side="right" # since pad_token = eos_token

In [ ]:
dataset = load_dataset(DATASET)
dataset = dataset.shuffle(seed=SEED)
# for faster training, we only train with 5000 rows of train data for training and 2000 rows of train data for evaluation
dataset["train"] = dataset["train"].train_test_split(train_size=TRAIN_SIZE, test_size=EVAL_SIZE, seed=SEED) 

print(f"Train dataset size: {len(dataset['train']['train'])} rows.")
print(f"Eval dataset size: {len(dataset['train']['test'])} rows.")
print(f"Test dataset size: {len(dataset['test'])} rows.")
print(dataset)

### **2-2. Data Visualization**
- the maximum tokenized `sql schema` & `sql code` length is **724** which is much smaller than the context length of `Gemma-2-2b-it` is **8192** tokens

In [ ]:
# Input (sql prompt + sql context)
tokenized_inputs = concatenate_datasets([dataset["train"]["train"], dataset["train"]["test"], dataset["test"]]).\
    map(lambda x: tokenizer(x["sql_context"]), batched=True)
tokenized_input_lengths = [len(x) for x in tokenized_inputs["input_ids"]]

# Output
tokenized_outputs = concatenate_datasets([dataset["train"]["train"], dataset["train"]["test"], dataset["test"]]).\
    map(lambda x: tokenizer(x["sql"]), batched=True)
tokenized_output_lengths = [len(x) for x in tokenized_outputs["input_ids"]]

# Console log
print(f"maximum tokenized input (sql schema) length + maximum tokenized output (sql code) length = {max(tokenized_input_lengths) + max(tokenized_output_lengths)}")

# Histogram
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].hist(tokenized_input_lengths, bins=20, color="#19334d")
axes[0].set_xlabel("tokenized_sql_schema_length")
axes[0].set_ylabel("times") 
axes[0].set_title("distribution of tokenized sql schema length") 

axes[1].hist(tokenized_output_lengths, bins=20, color="#669900")
axes[1].set_xlabel("tokenized_sql_code_length")
axes[1].set_ylabel("times")  
axes[1].set_title("distribution of tokenized sql code length") 

plt.tight_layout()

plt.show()

### **2-3. Data Preprocess**

- Use chat template to perform supervised fine-tuning on Gemma-2-2b-it

**reference**
- [LLM Fine-tuning Chat Template](https://github.com/mst272/LLM-Dojo/tree/main/chat_template#gemma)
- [Finetune Gemma-2b for Text to SQL](https://medium.com/@hayagriva99999/finetune-gemma-2b-for-text-to-sql-90041abdda70)

In [ ]:
# generate chat template
def generate_chat_template(sample):
    system_prompt = "You are a helpful assistant specialised on text-to-SQL. Given a question and context regarding one or more tables in database, your task is to generate a SQL query to answer questions."
    if sample['sql_context']:
        chat_template = f"""<start_of_turn>user {system_prompt}
SQL Question: {sample['sql_prompt']}
SQL Context: {sample['sql_context']}
Generated SQL Query: <end_of_turn>

<start_of_turn>model {sample['sql']} <end_of_turn>"""
    else:
        chat_template = f"""<start_of_turn>user {system_prompt}
SQL Question: {sample['sql_prompt']}
Generated SQL Query: <end_of_turn>

<start_of_turn>model {sample['sql']} <end_of_turn>"""
    return chat_template
    
# add prompt column to dataset
train_prompt_column = [generate_chat_template(sample) for sample in dataset["train"]["train"]]
eval_prompt_column = [generate_chat_template(sample) for sample in dataset["train"]["test"]]
dataset["train"]["train"] = dataset["train"]["train"].add_column("prompt", train_prompt_column)
dataset["train"]["test"] = dataset["train"]["test"].add_column("prompt", eval_prompt_column)

# preview
dataset

In [ ]:
# tokenize prompt
def data_preprocess(sample):
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side="right" # since pad_token = eos_token
    return tokenizer(sample["prompt"])

train_tokenized_data = dataset["train"]["train"].map(data_preprocess, batched=True, remove_columns=REMOVE_COLS)
eval_tokenized_data = dataset["train"]["test"].map(data_preprocess, batched=True, remove_columns=REMOVE_COLS)
test_tokenized_data = dataset["test"].map(data_preprocess, batched=True, remove_columns=REMOVE_COLS)

print("[TRAIN] tokenized_dataset:\n", train_tokenized_data)
print("[EVAL]  tokenized_dataset:\n", eval_tokenized_data)
print("[TEST]  tokenized_dataset:\n", test_tokenized_data)

### **2-4. Create DataCollator For CausalLM**

In [ ]:
# Create DataCollator
"""
Use the end-of-sequence token as the padding token and set mlm=False. 
This will use the inputs as labels shifted to the right by one element.

# tokenizer.pad_token = tokenizer.eos_token:
Generally, pad_token is [PAD] by default and eos_token is [SEP] by default.
Therefore, if we set pad_token as eos_token, it means that we set [SEP] as our pad_token instead of [PAD].
The reason we implement this is to let the model deal with the padding part more consistently and let us get better result.
"""
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# data_collator

## **3. Fine-tune Gemma-2-2b-it**

**Goal : prevent `CUDA out of memory` error**
- Fine-tune method = **QLoRA (Quantization + LoRA)**

### **3-1. Load Model**
-  **Gemma-2-2b-it**
    - BitsAndBytesConfig setting reference: [Finetune Gemma-2b for Text to SQL](https://medium.com/@hayagriva99999/finetune-gemma-2b-for-text-to-sql-90041abdda70)

In [ ]:
# load model with HuggingFace Token
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, 
    device_map={"":0}, 
    quantization_config=quantization_config, 
    token=HF_TOKEN
)
model

### **3-2. Get PEFT Model with LoRA Adapter**

if we set the LoRA config as follow:
```json
{
    "r": 64,
    "lora_alpha": 64,
    "target_modules": ["gate_proj", "down_proj", "up_proj", "q_proj", "v_proj", "k_proj", "o_proj"],
    "lora_dropout": 0.05,
    "bias": "none",
    "task_type": TaskType.CAUSAL_LM
}
```

the trainable parameters are as follow:

- **trainable params: 83,066,880 || all params: 2,697,408,768 || trainable%: 3.0795**

In [ ]:
"""
Overview of the supported task types:
- SEQ_CLS: Text classification.
- SEQ_2_SEQ_LM: Sequence-to-sequence language modeling.
- CAUSAL_LM: Causal language modeling.
- TOKEN_CLS: Token classification.
- QUESTION_ANS: Question answering.
- FEATURE_EXTRACTION: Feature extraction. Provides the hidden states which can be used as embeddings or features
  for downstream tasks.
"""

# Define LoRA Config
lora_config = LoraConfig(
     r=LORA_RANK,
     lora_alpha=LORA_ALPHA,
     target_modules=LORA_TARGET_MODULES,
     lora_dropout=LORA_DROPOUT,
     bias="none",
     task_type=TaskType.CAUSAL_LM
)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

### **3-3. Define Training Arguments**

In [ ]:
# Define training arguments (SFTConfig)
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    optim=OPTIMIZER,
    learning_rate=LEARNING_RATE,
    warmup_ratio = WARMUP_RATIO,
    gradient_accumulation_steps=GRADIENT_ACCUMULUATION_STEPS,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    logging_steps=LOGGING_STEPS,
    save_strategy=SAVE_STRATEGY,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,
    max_grad_norm=MAX_GRAD_NORM,
    max_steps=MAX_STEPS,
    report_to="none",
    dataset_text_field="prompt",
)

# Create Trainer Instance
torch.cuda.empty_cache()
tokenizer.padding_side = "right"
trainer = SFTTrainer(
    model=model,
    args=training_args,
    peft_config=lora_config,
    data_collator=data_collator,
    train_dataset=train_tokenized_data,
    eval_dataset=eval_tokenized_data
)

model.config.use_cache = False  # not use cache while training

### **3-4. Start Training**

In [ ]:
# train model
trainer.train()

### **3-5. Merge and Save Finetuned Model & Tokenizer to Disk**
- Merge the base model with LoRA weights
    - code reference: [Finetune Gemma-2b for Text to SQL](https://medium.com/@hayagriva99999/finetune-gemma-2b-for-text-to-sql-90041abdda70)

In [ ]:
# Save our LoRA model & tokenizer results
trainer.model.save_pretrained(PEFT_MODEL_ID)

In [ ]:
# Merge the model with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cpu", # since merge_and_unload() might perform weird when device_map={"":0}, we turned into use device_map="cpu"
    token=HF_TOKEN
)
finetuned_model = PeftModel.from_pretrained(base_model, "/kaggle/input/lora-text2sql")
finetuned_model = finetuned_model.merge_and_unload()

# Save the merged model
finetuned_model.save_pretrained(NEW_MODEL, safe_serialization=True)
tokenizer.save_pretrained(NEW_MODEL)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# delete models we don't need to reduce memory usage
import gc
del base_model
del finetuned_model
del tokenizer
gc.collect()

## **4. Evaluate the performance of LoRA fine-tuned CodeGemma-1.1-2b**

### **4-1. Load base model, tokenizer, fine-tuned model, and Test Dataset**

In [ ]:
# load two tokenizers (since they are on different devices)
base_tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
base_tokenizer.pad_token = base_tokenizer.eos_token
base_tokenizer.padding_side="right" # since pad_token = eos_token
text2sql_tokenizer = AutoTokenizer.from_pretrained(NEW_MODEL, token=HF_TOKEN)

# load base model
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"":0}, 
    token=HF_TOKEN
)

# load fine-tuned model
text2sql_model = AutoModelForCausalLM.from_pretrained(NEW_MODEL, token=HF_TOKEN)
text2sql_model = text2sql_model

# load test dataset from the hub
dataset = load_dataset(DATASET)
test_dataset = dataset["test"]
test_ids = random.sample(range(0, test_dataset.num_rows), TEST_CASES_NUMBER)

# print status quo
print("base tokenizer, finetuned tokenizer, base model, finetuned model and test dataset are all loaded...done.")
print("test dataset :")
print(test_dataset)

### **4-2. Define Evaluate Function**

In [ ]:
# define function for evaluate the performance of transforming text to sql
def evaluate(mode: str, sample: dict) -> (List[int], List[str]):
    # mode can only be "base" or "text2sql"
    assert mode in ["base", "text2sql"]

    # generate chat template
    system_prompt = "You are a helpful assistant specialised on text-to-SQL. Given a question and context regarding one or more tables in database, your task is to generate a SQL query to answer questions."
    if sample['sql_context']:
        chat_template = f"""<start_of_turn>user {system_prompt}
SQL Question: {sample['sql_prompt']}
SQL Context: {sample['sql_context']}
Generated SQL Query: <end_of_turn>

<start_of_turn>model 

"""
    else:
        chat_template = f"""<start_of_turn>user {system_prompt}
SQL Question: {sample['sql_prompt']}
Generated SQL Query: <end_of_turn>

<start_of_turn>model 

"""

    if mode == "base":
         # tokenize chat template
        model_inputs = base_tokenizer(chat_template, return_tensors="pt", add_special_tokens=True).to("cuda")
        input_length = model_inputs.input_ids.shape[1]
        # generate results with base model
        base_output_ids = base_model.generate(input_ids=model_inputs.input_ids, 
                                              max_new_tokens=MAX_NEW_TOKENS, 
                                              do_sample=True, 
                                              top_p=TOP_P,
                                              top_k=TOP_K,
                                              temperature=TEMPERATURE
                                             )
        output = base_tokenizer.decode(base_output_ids[0][input_length:].detach().cpu().numpy(), skip_special_tokens=True)

    if mode == "text2sql":
        # tokenize chat template
        model_inputs = text2sql_tokenizer(chat_template, return_tensors="pt", add_special_tokens=True).to("cpu")
        input_length = model_inputs.input_ids.shape[1]
        # generate results with finetuned text2sql model
        text2sql_output_ids = text2sql_model.generate(input_ids=model_inputs.input_ids, 
                                              max_new_tokens=MAX_NEW_TOKENS, 
                                              do_sample=True, 
                                              top_p=TOP_P,
                                              top_k=TOP_K,
                                              temperature=TEMPERATURE
                                             )
        output = text2sql_tokenizer.decode(text2sql_output_ids[0][input_length:].detach().cpu().numpy(), skip_special_tokens=True)

    # post process
    output = output.replace("\n", "").replace("```sql", "").replace("```", "").strip()
        
    # ground truth
    ground_truth = sample["sql"].strip()
    
    # print the result
    print(f"""Question:
>>> {sample['sql_prompt']}
Ground Truth Answer:
>>> {ground_truth}
{mode.capitalize()} Model Answer:
>>> {output}""")
    print()
    
    # output schema, query, ground truth answer, base model/text2sql model answer
    row_answer = [sample['sql_context'], sample['sql_prompt'], ground_truth, output]
    
    # check and calculate the accuracy
    print("Result:")
    if output == ground_truth:
        print(f"- {mode.capitalize()} Model Success.")
        result = 1
    else:
        print(f"- {mode.capitalize()} Model Failed.")
        result = 0
    return result, row_answer


### **4-3. (Optional) Test if our fine-tuned text-to-sql model works**

In [ ]:
# get one sample
mode = "text2sql"
sample = test_dataset[0]
result, row_answer = evaluate(mode, sample)
print("-"*30)
print(f"[{mode.capitalize()} Model] result & row_answer are as follows:")
print(result)
print(row_answer)

### **4-4. Evaluate the performance of the base model**
- `Gemma-2-2b-it`

In [ ]:
# control reproducibility
random.seed(SEED)

# evaluate and calculate accuracy(%)
base_model_logs = []
base_success_or_failed = []
for idx in tqdm(test_ids):
    sample = test_dataset[idx]
    result, row_answer = evaluate("base", sample)
    base_model_logs.append(row_answer)
    base_success_or_failed.append(result)
    
# compute accuracy
base_accuracy = sum(base_success_or_failed)/len(base_success_or_failed)

### **4-5. Evaluate the performance of our finetuned model**
- `LoRA Fine-tuned Text-to-SQL Gemma-2-2b-it`

In [ ]:
# control reproducibility
random.seed(SEED)

# evaluate and calculate accuracy(%)
text2sql_model_logs = []
text2sql_success_or_failed = []
for idx in tqdm(test_ids):
    sample = test_dataset[idx]
    result, row_answer = evaluate("text2sql", sample)
    text2sql_model_logs.append(row_answer)
    text2sql_success_or_failed.append(result)

# compute accuracy
text2sql_accuracy = sum(text2sql_success_or_failed)/len(text2sql_success_or_failed)

### **4-6. Print Test Result & Save As Excel File for SOTA LLM Judging**

In [ ]:
print(f"""

ACCURACY RESULT BY CHECKING IF THE SQL CODE IS THE SAME AS THE GROUND TRUTH SQL CODE
---------------------------------------------------
| [Base Model]     Text-to-SQL Accuracy: {base_accuracy*100:.2f}% |
| [Text2sql Model] Text-to-SQL Accuracy: {text2sql_accuracy*100:.2f}% |
---------------------------------------------------""")

In [ ]:
# base df
base_df = pd.DataFrame(base_model_logs, columns=[
    "Table Schema", "Question", "Ground Truth SQL Code", "Base Model SQL Code"
])

# text2sql df
text2sql_df = pd.DataFrame(text2sql_model_logs, columns=[
    "Table Schema", "Question", "Ground Truth SQL Code", "Text2sql Model SQL Code"
])

# filename
base_fname = f"text2sql_test_result_of_base_model_v{VERSION}.xlsx"
text2sql_fname = f"text2sql_test_result_of_text2sql_model_v{VERSION}.xlsx"

# output excel
base_df.to_excel(base_fname, index=False)
text2sql_df.to_excel(text2sql_fname, index=False)
print(f"test results are already saved.")